In [5]:
import os
import sys
import json
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.__version__

'2.2.0+cpu'

In [7]:
# print(sys.executable)
# print(os.getcwd())
# os.pardir

In [8]:
%matplotlib

Using matplotlib backend: <object object at 0x0000023296AA2120>


In [9]:
from PIL import Image
from matplotlib import pyplot as plt


In [10]:
model = torch.hub.load( 'pytorch/vision:v0.10.0', 'resnet18', pretrained=True )

model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model

Using cache found in C:\Users\Alex/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Alex\Documents\GB\DE_DC_and_M_2023\venv_cource_de_ad_and_m_2023\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
from torchvision import transforms

In [12]:
def class_of_picture(img):
    norm = transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] )

    inv_norm = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225])

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        norm,
    ])

    image_tensor = preprocess(img)
    input_tensor = image_tensor.unsqueeze(0) # Модель ожидает одно изображение ("один батч")
    input_tensor = input_tensor.to(device) # Отправим тензор на выбранное устройство (CPU/GPU/TPU)

    outputs = model( input_tensor )
    print('outputs.device == {}'.format(outputs.device))
    prediction = outputs.max(dim=1).indices.item()
    return prediction

In [13]:
imagenet_labels_file = './imagenet_labels.json'

In [14]:
img_file_name = './bernd-dittrich-6uXEUfLsoLc-unsplash.jpg'
usr_img = Image.open(img_file_name)

# plt.imshow(img);

if not ("IMAGENET_LABELS" in locals() and len(IMAGENET_LABELS) == 1000):
    with open(imagenet_labels_file, 'r') as f:
        IMAGENET_LABELS = json.load(f)

# Получите его класс
result = class_of_picture(usr_img)

print('ResNet50 prediction: {}'.format(IMAGENET_LABELS[str(result)]))

outputs.device == cpu
ResNet50 prediction: nail
